In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 11.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.4 MB/s eta 0:00

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate
from huggingface_hub import notebook_login
from transformers import pipeline

In [3]:
url_of_dataset = "https://raw.githubusercontent.com/LokasWiki/public-datasets/main/Datasets/usernames_spam.csv"
dataset = load_dataset("csv",data_files=url_of_dataset,split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-449c9a21a33baeb8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [4]:
dataset[0:10]

{'Name': ['Yousef10166',
  'توفيق الشارني',
  'Eng.salman1',
  'Moulay nadjem ALLOUAOUI',
  'Mmaarwa111',
  'Abdouflih99',
  'Tk8.m',
  'Marounbm',
  'Ibn ElKhtab',
  'Abdulla rashed'],
 'Category': ['ham',
  'ham',
  'ham',
  'ham',
  'ham',
  'ham',
  'ham',
  'ham',
  'ham',
  'ham']}

In [5]:
dataset = dataset.train_test_split(test_size=0.2)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Name', 'Category'],
        num_rows: 36419
    })
    test: Dataset({
        features: ['Name', 'Category'],
        num_rows: 9105
    })
})

In [7]:
dataset = dataset.rename_column("Category", "labels")
dataset = dataset.rename_column("Name", "text")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 36419
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 9105
    })
})

In [9]:
def cast_labels_to_int(example):
    example["labels"] = int(example["labels"] == "spam")
    return example


In [10]:
dataset = dataset.map(cast_labels_to_int)


Map:   0%|          | 0/36419 [00:00<?, ? examples/s]

Map:   0%|          | 0/9105 [00:00<?, ? examples/s]

In [11]:
dataset["train"][1]

{'text': 'ARIF ASHRAF', 'labels': 0}

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [14]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/36419 [00:00<?, ? examples/s]

Map:   0%|          | 0/9105 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [16]:
tokenized_datasets.set_format("torch")

In [17]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [18]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [19]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [20]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [21]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


In [22]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [23]:
progress_bar = tqdm(range(num_training_steps))

model.train()


  0%|          | 0/375 [00:00<?, ?it/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [24]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


In [25]:

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.876}

In [27]:
model.save_pretrained("/content/test")
tokenizer.save_pretrained("/content/test")

('/content/test/tokenizer_config.json',
 '/content/test/special_tokens_map.json',
 '/content/test/vocab.txt',
 '/content/test/added_tokens.json',
 '/content/test/tokenizer.json')

In [29]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [30]:
model.push_to_hub("spam-usernames-classifier")


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lokas/spam-usernames-classifier/commit/e72ca3eae5d76bddd4bde6a8baf0df3e684ab887', commit_message='Upload BertForSequenceClassification', commit_description='', oid='e72ca3eae5d76bddd4bde6a8baf0df3e684ab887', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
tokenizer.push_to_hub("spam-usernames-classifier")

CommitInfo(commit_url='https://huggingface.co/lokas/spam-usernames-classifier/commit/b23e39823dc6877b97dc7faeb36b672cd3bc3365', commit_message='Upload tokenizer', commit_description='', oid='b23e39823dc6877b97dc7faeb36b672cd3bc3365', pr_url=None, pr_revision=None, pr_num=None)